In [34]:
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.3f' % x)

from fsutils import run_sf_sql as rp, config, email, adls_gen2, log
from snowflake.connector.pandas_tools import write_pandas

# Fetch configs
configmapname = "notebook-xlarge"
params=config.get_config(configmapname)

# Get the SF connection and cursor
conn, cur=rp.get_connection(configmapname)

In [35]:
planogram_master = 'DL_FSCA_SLFSRV.TWA07.C890868_LOCALIZATION_POG_MASTER_0829_SALES_52W_202535_FULL_SCOPE'
cluster_table = 'DL_FSCA_SLFSRV.TWA07.ASSORTMENT_STORE_CLUSTERS_LATEST_10_14'

In [36]:
sql_query = f"""
    
    SELECT DISTINCT PLANO_CAT_DSC, PLANOGRAM_DSC, STORE_NBR, PLANOGRAM_NBR, 
        CAST(PLANOGRAM_VERSION_ID AS INTEGER) AS PLANOGRAM_VERSION_ID,
        VOLUME_FLAG, RISK_FLAG,
        CASE
            WHEN PLANOGRAM_NBR = 9150 AND TIME_ZONE_ID IN ('Mountain AZ', 'Mountain', 'Hawaii-Aleutian', 'Pacific', 'Alaska') THEN 'west'
            WHEN PLANOGRAM_NBR = 9150 AND TIME_ZONE_ID IN ('Eastern', 'Central', 'Atlantic-PR') THEN 'nowest'
            ELSE NULL
        END AS INCLUDED_REGIONS
    FROM {planogram_master}
    LEFT JOIN DL_FSCA_SLFSRV.TWA07.CLUSTERING_STORES_REGION_REMAPPING
    USING (STORE_NBR)
    WHERE (PLANO_CAT_DSC, store_nbr) in (SELECT DISTINCT cat_dsc, "store_nbr" FROM {cluster_table})
"""
df_store_planogram_version_mapping = conn.cursor().execute(sql_query).fetch_pandas_all()
df_store_planogram_version_mapping.columns = map(str.lower, df_store_planogram_version_mapping.columns)

In [37]:
# Load the Excel file
excel_file = pd.ExcelFile("data/Raw Input Data - Combo Versions 10.14.xlsx")
sheet_data = []

# Loop through each sheet
for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)
    df = df.rename(columns={
        "Group": "planogram_nbr",
        "Vers": "planogram_version_id",
        "Version": "planogram_version_id",
        "Included Group #’s": "included_planogram_nbr",
        "Included Group #'s": "included_planogram_nbr",
        "Included Groups": "included_planogram_nbr"
    })
    
    # Get relevant columns and drop entries with missing data
    df = df[["planogram_nbr", "planogram_version_id", "included_planogram_nbr"]].dropna()
    df = df.astype({
        "planogram_nbr": "int",
        "planogram_version_id": "int",
        "included_planogram_nbr": "str"
    })
    
    # Append to a list
    sheet_data.append(df)

# Load merch sif Excel file
excel_file = pd.ExcelFile("data/merch_aid.xlsx")
merch_aid_sheet_data = []

# Loop through each sheet
for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)
    df = df.rename(columns={
        "PLANO_NBR": "planogram_nbr",
        "STORE_NBR": "store_nbr",
        "FIXTURE_TYPE": "merch_aid_fixture_type",
        "ITEM_SCOPE": "item_scope"
    })
    
    # Get relevant columns and drop entries with missing data
    df = df[["planogram_nbr", "store_nbr", "merch_aid_fixture_type", "item_scope"]].dropna()
    df = df.astype({
        "planogram_nbr": "int",
        "store_nbr": "int",
        "merch_aid_fixture_type": "str",
        "item_scope": "str"
    })
    
    # Append to a list
    merch_aid_sheet_data.append(df)
   
df_combo_versions = pd.concat(sheet_data, ignore_index=True) 
df_merch_aid = pd.concat(merch_aid_sheet_data, ignore_index=True)

# For merch aid, if a store is in both FMBB & 3TIER, only keep FMBB
duplicate_store_list = df_merch_aid[df_merch_aid.duplicated(subset="store_nbr")]["store_nbr"].unique()
df_merch_aid = df_merch_aid[~((df_merch_aid["store_nbr"].isin(duplicate_store_list)) & (df_merch_aid["merch_aid_fixture_type"].isin(["FM571", "FMBB"])))]


df_combo_versions = pd.concat(sheet_data, ignore_index=True) 
df_merch_aid = pd.concat(merch_aid_sheet_data, ignore_index=True)

# For merch aid, if a store is in both FMBB & 3TIER, only keep FMBB
duplicate_store_list = df_merch_aid[df_merch_aid.duplicated(subset="store_nbr")]["store_nbr"].unique()
df_merch_aid = df_merch_aid[~((df_merch_aid["store_nbr"].isin(duplicate_store_list)) & (df_merch_aid["merch_aid_fixture_type"].isin(["FM571", "FMBB"])))]

# Merge planogram info
df_store_combo_versions = df_store_planogram_version_mapping.merge(
    df_combo_versions,
    on=["planogram_nbr", "planogram_version_id"],
    how="left"
)

# Merge merch aid info
df_store_combo_versions = df_store_combo_versions.merge(
    df_merch_aid,
    on=["planogram_nbr", "store_nbr"],
    how="left"
)

# Fill missing fields
df_store_combo_versions["included_planogram_nbr"] = df_store_combo_versions["included_planogram_nbr"].fillna("")
df_store_combo_versions["included_regions"] = df_store_combo_versions["included_regions"].fillna("")
df_store_combo_versions["merch_aid_fixture_type"] = df_store_combo_versions["merch_aid_fixture_type"].fillna("")

# concat fields
df_store_combo_versions["special_version_dsc"] = ( 
    df_store_combo_versions["included_planogram_nbr"] + ", " + df_store_combo_versions["included_regions"] 
)
df_store_combo_versions["special_version_dsc"] = (
 df_store_combo_versions["special_version_dsc"].str.strip(" ").str.strip(",").str.strip(" ").str.strip(",")  
)

# version components
df_store_combo_versions["special_version_components"] = ( 
    df_store_combo_versions["included_planogram_nbr"] 
    + "," 
    + df_store_combo_versions["included_regions"] 
    + ","
    + df_store_combo_versions["merch_aid_fixture_type"] 
)

# Replace empty stirng with nan
df_store_combo_versions["included_planogram_nbr"] = df_store_combo_versions["included_planogram_nbr"].replace("", np.nan)
df_store_combo_versions["included_regions"] = df_store_combo_versions["included_regions"].replace("", np.nan)
df_store_combo_versions["special_version_dsc"] = df_store_combo_versions["special_version_dsc"].replace("", np.nan)
df_store_combo_versions["merch_aid_fixture_type"] = df_store_combo_versions["merch_aid_fixture_type"].replace("", np.nan)

# Split into dataframe with special version vs not
df_normal_version = df_store_combo_versions[(df_store_combo_versions["included_planogram_nbr"].isna())
                        & (df_store_combo_versions["included_regions"].isna())
                        & (df_store_combo_versions["merch_aid_fixture_type"].isna())]
df_store_combo_versions = df_store_combo_versions[(df_store_combo_versions["included_planogram_nbr"].notna())
                        | (df_store_combo_versions["included_regions"].notna())
                        | (df_store_combo_versions["merch_aid_fixture_type"].notna())]

# For normal version, set version to 0
df_normal_version["special_version_id"] = 0

# For special version, add incremental version number
df_store_combo_versions = df_store_combo_versions.sort_values(["special_version_components"])
df_store_combo_versions["special_version_id"] = df_store_combo_versions.groupby(["planogram_nbr"])["special_version_components"].rank(method="dense").astype(int)

# Concat the two dataframes
df_store_combo_versions = pd.concat([df_store_combo_versions, df_normal_version], ignore_index=True)

# keep relevant columns
df_store_combo_versions_output = df_store_combo_versions[["plano_cat_dsc", "planogram_dsc", "planogram_nbr", 
                                                   "planogram_version_id", "special_version_dsc", 
                                                    "merch_aid_fixture_type", "special_version_id",
                                                   "volume_flag", "risk_flag",
                                                   "store_nbr"]].drop_duplicates()

# Check that each store planogram number is linked to version id
max_entries = (
    df_store_combo_versions_output
    .groupby(["store_nbr", "planogram_nbr"])
    ["special_version_id"]
    .nunique()
    .reset_index()
    ["special_version_id"]
    .max()
)

assert (max_entries == 1
       ), f"Expected 1 entry per store & planogram combo for special version. Found instances with {max_entries} entries"

df_store_combo_versions_output.columns = map(str.upper, df_store_combo_versions_output.columns)

In [38]:
write_pandas(conn, df_store_combo_versions_output, "STORE_SPECIAL_COMBO_VERSION_V1016", 
                   database="DL_FSCA_SLFSRV", 
                   schema="TWA07",
                   overwrite=True,
                   auto_create_table=True
            )

/home/jovyan/.local/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:344: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


(True,
 1,
 612551,
 [('acwkscubzm/file0.txt',
   'LOADED',
   612551,
   612551,
   1,
   0,
   None,
   None,
   None,
   None)])

In [51]:
sql_query = f"""
    SELECT DISTINCT PLANOGRAM_NBR AS INCLUDED_PLANOGRAM, SKU_NBR
    FROM {planogram_master}
    WHERE PLANO_CAT_DSC = (CASE WHEN CAT_DSC = 'LAUNDRY' THEN 'HOUSEHOLD'
                                WHEN CAT_DSC = 'SODA' THEN 'BEVERAGES'
                                ELSE CAT_DSC
                                END)
    UNION 

    SELECT DISTINCT POG_NUMBER AS INCLUDED_PLANOGRAM, UPC as SKU_NBR
    FROM DL_FSCA_SLFSRV.TWA07.FS_MANUAL_NEW_SKUS_1
"""
df_planogram_items = conn.cursor().execute(sql_query).fetch_pandas_all()
df_planogram_items.columns = map(str.lower, df_planogram_items.columns)

In [53]:
grocery_exclude_nowest = [495729, 455325, 924893, 243773, 492490, 355426, 468431, 500431, 613693, 754887, 501651, 939094, 500434, 883334, 522573, 335314, 993905, 500435, 500433, 354513, 939095, 188566] 


# Load the Excel file
excel_file = pd.ExcelFile("data/item_sub_plano.xlsx")
sheet_data = []

# Loop through each sheet
for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)
    df = df.rename(columns={
        "SKU_NBR": "sku_nbr",
        "PLANOGRAM_NBR": "planogram_nbr",
        "SUB_PLANO": "sub_plano"
    })
    
    # Get relevant columns and drop entries with missing data
    df = df[["sku_nbr", "planogram_nbr", "sub_plano"]].dropna()
    df = df.astype({
        "sku_nbr": "int",
        "planogram_nbr": "int",
        "sub_plano": "str"
    })
    
    # Append to a list
    sheet_data.append(df)

df_sub_plano = pd.concat(sheet_data, ignore_index=True)

In [54]:
# Load the Excel file
excel_file = pd.ExcelFile("data/Combo Pogs Override - Template.xlsx")
sheet_data = []

# Loop through each sheet
for sheet_name in excel_file.sheet_names:
    df = excel_file.parse(sheet_name)
    
    # Get relevant columns and drop entries with missing data
    df = df[["planogram_nbr", "sku_nbr", "included_planogram"]].dropna()
    
    # Append to a list
    sheet_data.append(df)

df_combo_pogs_override = pd.concat(sheet_data, ignore_index=True)

In [55]:
df_combo_item_scope = df_store_combo_versions.copy()
df_plano_cat_nbr_map = df_combo_item_scope[["plano_cat_dsc", "planogram_nbr"]].drop_duplicates()

df_combo_item_scope = df_combo_item_scope[["planogram_nbr", "special_version_id", "included_regions", 
                                           "merch_aid_fixture_type", "included_planogram_nbr", "item_scope"]].drop_duplicates()
df_combo_item_scope["included_planogram_nbr"] = (
    df_combo_item_scope["included_planogram_nbr"].fillna("") + "," + df_combo_item_scope["planogram_nbr"].astype(str)
) 


# Make each entry a row
df_combo_item_scope = df_combo_item_scope[(df_combo_item_scope["included_planogram_nbr"] != " ") & (df_combo_item_scope["included_planogram_nbr"] != "")]
df_combo_item_scope = df_combo_item_scope[df_combo_item_scope["included_planogram_nbr"] != ""]
df_combo_item_scope = df_combo_item_scope.assign(included_planogram=df_combo_item_scope["included_planogram_nbr"].str.split(",")).explode("included_planogram")
df_combo_item_scope = df_combo_item_scope[(df_combo_item_scope["included_planogram"] != " ") & (df_combo_item_scope["included_planogram"] != "")]
df_combo_item_scope = df_combo_item_scope.drop(columns=["included_planogram_nbr"])

# Adjust column types
df_combo_item_scope["included_planogram"] = df_combo_item_scope["included_planogram"].astype(float)
df_planogram_items["included_planogram"] = df_planogram_items["included_planogram"].astype(float)

# Merge items in scope
df_combo_item_scope = df_combo_item_scope.merge(
    df_planogram_items,
    on="included_planogram",
    how="left"
)

# Drop rows were sku is missing
df_combo_item_scope = df_combo_item_scope[df_combo_item_scope["sku_nbr"].notna()]
df_combo_item_scope["sku_nbr"] = df_combo_item_scope["sku_nbr"].astype(int)

# Remove items not eligble in nowest
df_combo_item_scope = df_combo_item_scope.loc[~(
    (df_combo_item_scope["included_regions"] == "nowest")
    & (df_combo_item_scope["sku_nbr"].isin(grocery_exclude_nowest))
)].drop(columns=["included_regions"])

In [56]:
## remove all planogram_nbr X sku_nbr combination in the override table
## only include those planogram_nbr X sku_nbr in a given special version
override_scope = df_combo_item_scope[["planogram_nbr","special_version_id","merch_aid_fixture_type","item_scope","included_planogram"]].drop_duplicates()\
                                    .merge(df_combo_pogs_override, on = ["planogram_nbr","included_planogram"],how='inner')

unaffected_scope = df_combo_item_scope.merge(df_combo_pogs_override[["planogram_nbr","sku_nbr"]].drop_duplicates(), on = ["planogram_nbr","sku_nbr"],how='left',indicator=True).query("_merge == 'left_only'")\
                    .drop(columns = ['_merge'])

df_combo_item_scope = pd.concat([override_scope,unaffected_scope])


In [57]:
# Shaving Needs, adult care Scope
# For version_id = 0, assume no subplano, but keep all items in scope
planos_remove_subplano = [9380, 9569, 9330]

if planos_remove_subplano:
    ### For shaving needs, each version only contains its only subplano SKUs
    non_sub_plano_item = df_combo_item_scope.query("planogram_nbr in @planos_remove_subplano").merge(
        df_sub_plano,
        on=["planogram_nbr","sku_nbr"],
        how="left",
        indicator = True
    ).pipe(lambda dx: dx[(dx["_merge"] == 'left_only') | dx["merch_aid_fixture_type"].isna()]).drop(columns = ["_merge"])\
    .assign(sub_plano = np.nan)
    
    sub_plano_item = df_combo_item_scope.query("planogram_nbr in @planos_remove_subplano").merge(
        df_sub_plano,
        on=["planogram_nbr","sku_nbr"],
        how="left"
    ).pipe(lambda dx: dx[(dx["merch_aid_fixture_type"].notna()) & (dx["item_scope"] == dx["sub_plano"])])
    
    df_combo_item_scope_shaving = pd.concat([non_sub_plano_item,sub_plano_item])

else:
    df_combo_item_scope_shaving = pd.DataFrame()

In [58]:
# Non-Shaving Needs Scope
### For non-shaving needs, keep the previous way
# For version_id = 0, assume it have all subplanos

# For items with special consideration, add to the planogram they belong to
df_unique_versions = df_combo_item_scope[["planogram_nbr", "special_version_id", "merch_aid_fixture_type", "included_planogram", "item_scope"]].drop_duplicates()

# Make each item_scope a row
df_unique_versions = df_unique_versions.assign(item_scope=df_unique_versions["item_scope"].str.split(",")).explode("item_scope")


df_unique_versions = df_unique_versions.merge(
    df_sub_plano,
    on=["planogram_nbr"],
    how="left"
)


# Drop items without a sub_plano
df_unique_versions = df_unique_versions[df_unique_versions["sub_plano"].notna()]

# Apply filters specific to merh aid SKUs
df_unique_versions = df_unique_versions[~(
    (df_unique_versions["planogram_nbr"].isin([9600])) # Filtering for face mask merch aid, shaving needs goodline
    & (df_unique_versions["merch_aid_fixture_type"].notna()) # Filter out entries not related to merch aid 
    & (df_unique_versions["item_scope"] != df_unique_versions["sub_plano"]) # Filtering out entries with mismatching scope/sub_plano
    )
]


# Get items with sub_plano consideration
df_unique_versions["sku_nbr"] = df_unique_versions["sku_nbr"].astype(int)
df_sub_plano_items = df_unique_versions[["planogram_nbr", "special_version_id", "sku_nbr"]].drop_duplicates()

# Filter out items from general dataframe if they exist in the dataframe of items with special consideration
df_combo_item_scope = df_combo_item_scope.merge(
    df_sub_plano_items,
    on=["planogram_nbr", "special_version_id", "sku_nbr"],
    how="left",
    indicator=True
)
df_combo_item_scope = df_combo_item_scope[df_combo_item_scope["_merge"] == "left_only"].drop(columns=["_merge"])


# Apply filter to exclude 
df_unique_versions = df_unique_versions[~(
    (df_unique_versions["planogram_nbr"].isin(planos_remove_subplano)) # Filtering for face mask merch aid, remove any planos has been adjusted
    & (df_unique_versions["merch_aid_fixture_type"].notna()) # Filter out entries not related to merch aid 
    & (df_unique_versions["item_scope"] != df_unique_versions["sub_plano"]) # Filtering out entries with mismatching scope/sub_plano
    )
]

# Concat the two dataframes
df_combo_item_scope_non_shaving = pd.concat([df_combo_item_scope, df_unique_versions], ignore_index=True).query("planogram_nbr not in @planos_remove_subplano")


df_combo_item_scope = pd.concat([df_combo_item_scope_shaving, df_combo_item_scope_non_shaving], ignore_index=True)


In [59]:
# Merge plano cat dsc info
df_combo_item_scope = df_combo_item_scope.merge(df_plano_cat_nbr_map, on="planogram_nbr", how="left")

# Keep relevant columns
df_combo_item_scope_output = df_combo_item_scope[["plano_cat_dsc", "planogram_nbr", "special_version_id", "sku_nbr", "sub_plano"]].drop_duplicates()

df_combo_item_scope_output.columns = map(str.upper, df_combo_item_scope_output.columns)

In [60]:
# df_combo_item_scope_output.query("PLANOGRAM_NBR == 9380").to_csv("testing.csv")

In [61]:
write_pandas(conn, df_combo_item_scope_output, "PLANOGRAM_VERSION_ITEM_SCOPE_V1017", 
                   database="DL_FSCA_SLFSRV",
                   schema="TWA07",
                   overwrite=True,
                   auto_create_table=True
            )

/tmp/ipykernel_4409/1836571257.py:1: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(conn, df_combo_item_scope_output, "PLANOGRAM_VERSION_ITEM_SCOPE_V1017",
/home/jovyan/.local/lib/python3.11/site-packages/snowflake/connector/pandas_tools.py:344: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  [pandas.api.types.is_datetime64tz_dtype(df[c]) for c in df.columns]


(True,
 1,
 145159,
 [('rzehvomcdo/file0.txt',
   'LOADED',
   145159,
   145159,
   1,
   0,
   None,
   None,
   None,
   None)])